In [ ]:
from os import listdir
from numpy import array
from keras import initializers
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adamax
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
from helpers.attention_decoder import AttentionDecoder

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
dir_name = 'resources/eval_light/'

# Read a file and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_data(data_dir):
    text = []
    images = []
    # Load all the files and order them
    all_filenames = listdir(data_dir)
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:] == "npz":
            # Load the images already prepared in arrays
            image = np.load(data_dir+filename)
            images.append(image['features'])
        else:
            # Load the boostrap tokens and rap them in a start and end tag
            syntax = '<START> ' + load_doc(data_dir+filename) + ' <END>'
            # Seperate all the words with a single space
            syntax = ' '.join(syntax.split())
            # Add a space after each comma
            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
    images = np.array(images, dtype=float)
    return images, text

train_features, texts = load_data(dir_name)

In [9]:
# Initialize the function to create the vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)
# Create the vocabulary 
tokenizer.fit_on_texts([load_doc('resources/bootstrap.vocab')])

# Add one spot for the empty word in the vocabulary 
vocab_size = len(tokenizer.word_index) + 1
# Map the input sentences into the vocabulary indexes
train_sequences = tokenizer.texts_to_sequences(texts)
# The longest set of boostrap tokens
max_sequence = max(len(s) for s in train_sequences)
# Specify how many tokens to have in each input sentence
max_length = 48

def preprocess_data(sequences, features):
    X, y, image_data = list(), list(), list()
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_sequence)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-48:])
            y.append(out_seq)
    return np.array(X), np.array(y), np.array(image_data)

X, y, image_data = preprocess_data(train_sequences, train_features)

In [11]:
print(vocab_size)
print(texts[0])
print(train_sequences[0])
print(max_sequence)

18
<START> header { btn-inactive , btn-active , btn-inactive , btn-inactive , btn-inactive } row { quadruple { small-title , text , btn-orange } quadruple { small-title , text , btn-red } quadruple { small-title , text , btn-green } quadruple { small-title , text , btn-orange } } row { single { small-title , text , btn-green } } <END>
[13, 14, 2, 8, 1, 15, 1, 8, 1, 8, 1, 8, 3, 7, 2, 6, 2, 4, 1, 5, 1, 9, 3, 6, 2, 4, 1, 5, 1, 11, 3, 6, 2, 4, 1, 5, 1, 10, 3, 6, 2, 4, 1, 5, 1, 9, 3, 3, 7, 2, 17, 2, 4, 1, 5, 1, 10, 3, 3, 16]
95


In [16]:
#Create the encoder
image_model = Sequential()
image_model.add(Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3,)))
image_model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(max_length))

visual_input = Input(shape=(256, 256, 3,))
encoded_image = image_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = LSTM(128, return_sequences=True)(language_model)

#Create the decoder
decoder = concatenate([encoded_image, language_model])
decoder = AttentionDecoder(100, 100)(decoder)
# model.add(AttentionDecoder(150, n_features))
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(vocab_size, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[visual_input, language_input], outputs=decoder)
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

inputs shape: (?, 48, 1152)


In [17]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [18]:
#Save the model for every 2nd epoch
filepath="org-weights-epoch-{epoch:04d}--val_loss-{val_loss:.4f}--loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=1)
# Tensorboard
# board = TensorBoard(log_dir='./logs', histogram_freq=10, batch_size=10, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
# callbacks_list = [checkpoint, board]

In [19]:
# Train the model
# model.fit([image_data, X], y, batch_size=10, shuffle=False, validation_split=0.1, callbacks=callbacks_list, verbose=1, epochs=50)
model.fit([image_data, X], y, batch_size=10, shuffle=False, validation_split=0.1, verbose=1, epochs=50)

Train on 624 samples, validate on 70 samples
Epoch 1/50
624/624 [==============================] - 50s 80ms/step - loss: 2.6394 - val_loss: 2.5175
Epoch 2/50
624/624 [==============================] - 44s 70ms/step - loss: 2.5023 - val_loss: 2.4792
Epoch 3/50
624/624 [==============================] - 44s 71ms/step - loss: 2.4742 - val_loss: 2.4554
Epoch 4/50
624/624 [==============================] - 44s 70ms/step - loss: 2.4515 - val_loss: 2.4370
Epoch 5/50
624/624 [==============================] - 44s 71ms/step - loss: 2.4328 - val_loss: 2.4208
Epoch 6/50
624/624 [==============================] - 44s 71ms/step - loss: 2.4160 - val_loss: 2.4070
Epoch 7/50
624/624 [==============================] - 44s 71ms/step - loss: 2.4012 - val_loss: 2.3947
Epoch 8/50
624/624 [==============================] - 44s 71ms/step - loss: 2.3883 - val_loss: 2.3829
Epoch 9/50
624/624 [==============================] - 43s 70ms/step - loss: 2.3769 - val_loss: 2.3715
Epoch 10/50
624/624 [================

KeyboardInterrupt: 